In [2]:
from dotenv import load_dotenv
import os
import boto3

# 加載 .env 文件
load_dotenv()

# 使用加載的環境變數來創建 boto3 客戶端
s3_client = boto3.client("s3")

# 列出所有 S3 buckets 作為測試
buckets = s3_client.list_buckets()
for bucket in buckets["Buckets"]:
    print(bucket["Name"])

bucket102501
elasticbeanstalk-us-east-1-114726445145


In [1]:
from dotenv import load_dotenv
import os
import boto3
from botocore.exceptions import ClientError

# 加載 .env 文件
load_dotenv()

# 使用加載的環境變數來創建 boto3 客戶端
ec2_client = boto3.client("ec2")

# 查詢並列出所有 EC2 實例
try:
    response = ec2_client.describe_instances()
    for reservation in response["Reservations"]:
        for instance in reservation["Instances"]:
            print(f"Instance ID: {instance['InstanceId']}")
            print(f"Instance Type: {instance['InstanceType']}")
            print(f"Instance State: {instance['State']['Name']}")
            print(f"Public IP: {instance.get('PublicIpAddress', 'N/A')}")
            print("-" * 20)
except ClientError as e:
    print(f"An error occurred: {e}")


In [3]:
import boto3
from botocore.exceptions import ClientError
import time

from dotenv import load_dotenv
import os
# 加載 .env 文件
load_dotenv()

# 使用本地環境的認證
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

# 設定資料表名稱
table_name = 'Table_20240718_01'

# 創建資料表
try:
    table = dynamodb.create_table(
        TableName=table_name,
        KeySchema=[
            {
                'AttributeName': 'movieId',
                'KeyType': 'HASH'  # 分區鍵
            }
        ],
        AttributeDefinitions=[
            {
                'AttributeName': 'movieId',
                'AttributeType': 'N'  # 數字型
            }
        ],
        ProvisionedThroughput={
            'ReadCapacityUnits': 5,
            'WriteCapacityUnits': 5
        }
    )
    print(f"創建資料表 {table_name} 成功。等待資料表變為 active 狀態...")
    table.meta.client.get_waiter('table_exists').wait(TableName=table_name)
    print("資料表已創建並可使用。")
except ClientError as e:
    print(e.response['Error']['Message'])

# 插入資料
table = dynamodb.Table(table_name)
try:
    table.put_item(
        Item={
            'movieId': 25,
            'title': 'Sample Movie Title',
            'genres': 'Action|Thriller'
        }
    )
    print("資料插入成功。")
except ClientError as e:
    print("資料插入失敗:", e.response['Error']['Message'])

# 查詢資料
try:
    response = table.get_item(
        Key={
            'movieId': 25
        }
    )
    item = response.get('Item')
    if item:
        print("查詢結果：", item)
    else:
        print("未找到該 movieId 的資料")
except ClientError as e:
    print("查詢失敗:", e.response['Error']['Message'])

創建資料表 Table_20240718_01 成功。等待資料表變為 active 狀態...
資料表已創建並可使用。
資料插入成功。
查詢結果： {'movieId': Decimal('25'), 'genres': 'Action|Thriller', 'title': 'Sample Movie Title'}


In [4]:
import boto3
from botocore.exceptions import ClientError

from dotenv import load_dotenv
import os
# 加載 .env 文件
load_dotenv()

# 使用本地環境的認證
dynamodb = boto3.resource('dynamodb', region_name='us-east-1')

# 設定要刪除的資料表名稱
table_name = 'Table_20240718_01'

# 刪除資料表
try:
    table = dynamodb.Table(table_name)
    table.delete()
    print(f"刪除資料表 {table_name} 的請求已提交，等待資料表刪除完成...")
    table.meta.client.get_waiter('table_not_exists').wait(TableName=table_name)
    print(f"資料表 {table_name} 已成功刪除。")
except ClientError as e:
    print("刪除資料表失敗:", e.response['Error']['Message'])

刪除資料表 Table_20240718_01 的請求已提交，等待資料表刪除完成...
資料表 Table_20240718_01 已成功刪除。
